# Liberías

In [16]:

import numpy as np
import pandas as pd

import sklearn
from sklearn.preprocessing import OneHotEncoder
from sklearn import preprocessing
from sklearn.metrics import classification_report,recall_score, precision_score
from sklearn.naive_bayes import GaussianNB
from sklearn import tree,svm,neighbors
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier,AdaBoostClassifier, RandomForestClassifier
from sklearn.neighbors import NearestCentroid
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV 
from sklearn.preprocessing import OrdinalEncoder

import matplotlib.pyplot as plt

from keras.layers import Dense, Dropout
from keras.models import Sequential
from tensorflow.keras.utils import to_categorical
from keras.wrappers.scikit_learn import KerasClassifier
from tensorflow.keras.optimizers import RMSprop
from keras.constraints import maxnorm

In [3]:
import tensorflow as tf
# from tf.compat.v1.keras.backend.set_session import set_session
config = tf.compat.v1.ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.4
config.gpu_options.allow_growth = True  # dynamically grow the memory used on the GPU
config.log_device_placement = True  # to log device placement (on which device the operation ran)
                                    # (nothing gets printed in Jupyter, only if you run it standalone)
sess = tf.compat.v1.Session(config=config)
tf.compat.v1.keras.backend.set_session(sess)

Device mapping: no known devices.


# Lectura de datos

In [17]:
data = pd.read_csv('../../splited_RASFF_DATA_16092019.csv', sep=';', 
                header=0, index_col = 0)
data ['DATE_CASE'] = data ['DATE_CASE'].astype(str)
data ['HAZARDS_CAT'] = data ['HAZARDS_CAT'].astype(str)
data = data.sample(frac = 1)
train  = data
train

,DATE_CASE,NOT_COUNTRY,PROD_CAT,TYPE,RISK_DECISION,ACTION_TAKEN,DISTRIBUTION_STAT,HAZARDS_CAT,COUNT_ORIGEN,COUNT_DESTIN,COUNT_CONCERN
54239,2015-07-08,Germany,"cocoa and cocoa preparations, coffee and tea",food,undecided,withdrawal from the market,distribution to other member countries,pesticide residues,China,Denmark,
76451,2013-03-18,Germany,"nuts, nut products and seeds",food,serious,return to consignor,distribution restricted to notifying country,mycotoxins,India,Germany,
118057,2008-01-29,United Kingdom,fruits and vegetables,food,undecided,(obsolete),no distribution,pathogenic micro-organisms,United States,Turks and Caicos Islands,United Kingdom
41546,2016-10-06,Belgium,compound feeds,feed,serious,withdrawal from the market,distribution to other member countries,environmental pollutants,Italy,Russia,Commission Services
95400,2011-03-07,Spain,herbs and spices,food,undecided,re-dispatch,no distribution,organoleptic aspects,Pakistan,,Spain
...,...,...,...,...,...,...,...,...,...,...,...
125534,2006-10-31,United Kingdom,fish and fish products,food,undecided,destruction,no distribution,environmental pollutants,Thailand,,United Kingdom
66669,2014-04-09,France,"dietetic foods, food supplements, fortified foods",food,serious,recall from consumers,distribution to other member countries,composition,United States,Ireland,Commission Services
140887,2004-02-12,Italy,"nuts, nut products and seeds",food,undecided,re-dispatch,,mycotoxins,Iran,,Italy
112541,2008-12-01,United Kingdom,crustaceans and products thereof,food,undecided,withdrawal from the market,distribution restricted to notifying country,residues of veterinary medicinal products,India,United Kingdom,Spain


# Features

In [18]:
features = [0,1,6,8]
target = [2]
X = train.iloc[:,features]
y = train.iloc[:,target]

In [19]:
X 

,DATE_CASE,NOT_COUNTRY,DISTRIBUTION_STAT,COUNT_ORIGEN
54239,2015-07-08,Germany,distribution to other member countries,China
76451,2013-03-18,Germany,distribution restricted to notifying country,India
118057,2008-01-29,United Kingdom,no distribution,United States
41546,2016-10-06,Belgium,distribution to other member countries,Italy
95400,2011-03-07,Spain,no distribution,Pakistan
...,...,...,...,...
125534,2006-10-31,United Kingdom,no distribution,Thailand
66669,2014-04-09,France,distribution to other member countries,United States
140887,2004-02-12,Italy,,Iran
112541,2008-12-01,United Kingdom,distribution restricted to notifying country,India


In [20]:
y

,PROD_CAT
54239,"cocoa and cocoa preparations, coffee and tea"
76451,"nuts, nut products and seeds"
118057,fruits and vegetables
41546,compound feeds
95400,herbs and spices
...,...
125534,fish and fish products
66669,"dietetic foods, food supplements, fortified foods"
140887,"nuts, nut products and seeds"
112541,crustaceans and products thereof


# Split

In [21]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

# Para probar codificaciones (opcional)

In [22]:
encX = OneHotEncoder(handle_unknown='ignore')
#encX = OrdinalEncoder()
#encX = FeatureHasher(n_features=25,input_type='string')
#encX = MultiLabelBinarizer()
encX.fit(X.values)
X_one_hot = encX.transform(X.values)

In [23]:
X_train, X_test, y_train, y_test = train_test_split(X_one_hot, y, test_size=0.2)

# Árboles de decisión

In [ ]:
params ={'criterion' :['gini', 'entropy'], 
         'splitter' :['best', 'random'], 
         'max_features':['auto', 'sqrt', 'log2']
        }
grid_search_cv = GridSearchCV(tree.DecisionTreeClassifier(random_state=42), params, verbose=1, cv=3)
clf = grid_search_cv.fit(X_train, y_train)

In [12]:
clf.best_estimator_

DecisionTreeClassifier(max_features='auto', random_state=42, splitter='random')

In [13]:
clf = tree.DecisionTreeClassifier(criterion='entropy', max_features='auto',
                       random_state=42)
clf = clf.fit(X_train,y_train)
y_pred = clf.predict(X_test)

In [14]:
print(classification_report(y_test, y_pred, zero_division=0))

                                                   precision    recall  f1-score   support

                              alcoholic beverages       0.89      0.86      0.88        86
                  animal by-products - (obsolete)       0.80      0.57      0.67         7
                    animal nutrition - (obsolete)       0.91      0.81      0.86       128
            bivalve molluscs and products thereof       0.83      0.76      0.79       492
                 cephalopods and products thereof       0.64      0.66      0.65       159
                      cereals and bakery products       0.84      0.80      0.82      1348
     cocoa and cocoa preparations, coffee and tea       0.81      0.88      0.84       615
                                   compound feeds       0.86      0.88      0.87       186
                                    confectionery       0.85      0.83      0.84       576
                 crustaceans and products thereof       0.72      0.73      0.73       51

In [15]:
print("specificity: ",clf.score(X_test, y_test))

print("sensitivity: ",recall_score(y_test, y_pred, average='macro'))

print("precision: ",precision_score(y_test, y_pred, average='macro'))

confusion_matrix(y_test, y_pred)

0.8505970465625663

# Random Forest

In [2]:
param_grid = {
    'bootstrap': [True],
    'max_depth': [80, 90, 100, 110],
    'max_features': [2, 3],
    'min_samples_leaf': [3, 4, 5],
    'min_samples_split': [8, 10, 12],
    'n_estimators': [100, 200, 300, 1000]}

rf = RandomForestClassifier()

grid_search = GridSearchCV(estimator = rf, param_grid = param_grid, 
                          cv = 3, n_jobs = -1, verbose = 2)
grid_search.fit(X_train, y_train)
grid_search.best_params_

NameError: name 'RandomForestClassifier' is not defined

In [25]:
clf = RandomForestClassifier(bootstrap = True, max_depth = 110, max_features = 3, min_samples_leaf =  3, min_samples_split = 8, n_estimators =  200)
clf = clf.fit(X_train,y_train)

<ipython-input-25-50ed6adac834>:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  clf = clf.fit(X_train,y_train)


In [26]:
y_pred = clf.predict(X_test)

In [27]:
y_pred

array(['fruits and vegetables', 'fruits and vegetables',
       'fruits and vegetables', ...,
       'dietetic foods, food supplements, fortified foods',
       'dietetic foods, food supplements, fortified foods',
       'dietetic foods, food supplements, fortified foods'], dtype=object)

In [28]:
print(classification_report(y_test,y_pred))

C:\Users\Enrique\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


                                                   precision    recall  f1-score   support

                              alcoholic beverages       0.00      0.00      0.00        86
                  animal by-products - (obsolete)       0.00      0.00      0.00         7
                    animal nutrition - (obsolete)       0.00      0.00      0.00       128
            bivalve molluscs and products thereof       0.00      0.00      0.00       492
                 cephalopods and products thereof       0.00      0.00      0.00       159
                      cereals and bakery products       1.00      0.02      0.04      1348
     cocoa and cocoa preparations, coffee and tea       0.00      0.00      0.00       615
                                   compound feeds       0.00      0.00      0.00       186
                                    confectionery       0.00      0.00      0.00       576
                 crustaceans and products thereof       0.00      0.00      0.00       51

C:\Users\Enrique\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Enrique\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [29]:
clf.score(X_test, y_test)

0.32816364021762173

# Gaussian naive Bayes clasiffier

In [ ]:
params = {'var_smoothing': np.logspace(0,-9, num=1)}
gs_NB =  GridSearchCV(GaussianNB(), params, verbose=1, cv=3)
gs_NB.fit(X_train.toarray(), y_train)

gs_NB.best_params_

In [ ]:
clf = GaussianNB(var_smoothing = 1.0)
clf = clf.fit(X_train_one_hot.toarray(),y_train)

In [ ]:
y_pred = clf.predict(X_test_one_hot.toarray())

In [ ]:
print(classification_report(y_test,y_pred))

In [ ]:
clf.score(X_test_one_hot.toarray(), y_test)

# Regresión logística

In [ ]:
clf = LogisticRegression()
grid_values = {'penalty': ['l2'],'C':[0.001,.009,0.01,.09,1,5,10,25]}
grid_clf_acc = GridSearchCV(clf, param_grid = grid_values,scoring = 'recall')
grid_clf_acc.fit(X_train, np.ravel(y_train))

In [24]:
grid_clf_acc.best_params_

{'C': 0.001, 'penalty': 'l2'}

In [25]:
clf = LogisticRegression(penalty= 'l2',C=0.001)
clf = clf.fit(X_train,y_train)

D:\Anaconda\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


In [26]:
y_pred = clf.predict(X_test)

In [27]:
print(classification_report(y_test,y_pred))

D:\Anaconda\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


                                                   precision    recall  f1-score   support

                              alcoholic beverages       0.00      0.00      0.00        83
                  animal by-products - (obsolete)       0.00      0.00      0.00        11
                    animal nutrition - (obsolete)       0.00      0.00      0.00       125
            bivalve molluscs and products thereof       0.00      0.00      0.00       461
                 cephalopods and products thereof       0.00      0.00      0.00       165
                      cereals and bakery products       0.37      0.03      0.06      1429
     cocoa and cocoa preparations, coffee and tea       0.00      0.00      0.00       622
                                   compound feeds       0.00      0.00      0.00       185
                                    confectionery       0.00      0.00      0.00       544
                 crustaceans and products thereof       0.00      0.00      0.00       52

D:\Anaconda\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
D:\Anaconda\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [28]:
clf.score(X_test, y_test)

0.32025012364869637

# Support vector machine

In [30]:
clf = svm.SVC()
param_grid = {'C': [0.1, 1, 10, 100, 1000], 
              'gamma': [1, 0.1, 0.01, 0.001, 0.0001],
              'kernel': ['rbf']} 
  
clf = GridSearchCV(clf, param_grid, refit = True, verbose = 3)
clf = clf.fit(X_train,y_train)
y_pred = clf.predict(X_test)
print(classification_report(y_test,y_pred))

Fitting 5 folds for each of 25 candidates, totalling 125 fits


D:\Anaconda\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


[CV 1/5] END .....................C=0.1, gamma=1, kernel=rbf; total time=12.6min


D:\Anaconda\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


[CV 2/5] END .....................C=0.1, gamma=1, kernel=rbf; total time=13.0min


...

D:\Anaconda\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


[CV 4/5] END ................C=0.1, gamma=0.0001, kernel=rbf; total time= 9.4min


D:\Anaconda\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


[CV 5/5] END ................C=0.1, gamma=0.0001, kernel=rbf; total time= 9.2min


D:\Anaconda\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


In [ ]:
clf.best_params_

In [ ]:
clf.score(X_test, y_test)

# Boosted Trees

In [11]:
param_grid = {'n_estimators':range(20,81,10),
 'learning_rate':[1, 0.5, 0.25, 0.1, 0.05, 0.01],
  'max_depth':range(5,16,2),
  'min_samples_split':range(200,1001,200)}

In [16]:
clf = GridSearchCV(estimator = GradientBoostingClassifier(max_features='sqrt', subsample=0.8, random_state=10), 
param_grid = param_grid,n_jobs=4, cv=3)
clf = clf.fit(X_train,y_train)
y_pred = clf.predict(X_test)
print(classification_report(y_test,y_pred))

In [ ]:
clf.score(X_test, y_test)

In [ ]:
clf = AdaBoostClassifier()
clf = clf.fit(X_train_one_hot,y_train)
y_pred = clf.predict(X_test_one_hot)
print(classification_report(y_test,y_pred))

In [ ]:
clf.score(X_test_one_hot, y_test)

# Nearest centroid

In [ ]:
clf = NearestCentroid()
clf = clf.fit(X_train_one_hot,y_train)
y_pred = clf.predict(X_test_one_hot)
print(classification_report(y_test,y_pred))

In [ ]:
clf.score(X_test_one_hot, y_test)

# Nearest neighbor

In [ ]:
clf = neighbors.KNeighborsClassifier()
clf = clf.fit(X_train_one_hot,y_train)
y_pred = clf.predict(X_test_one_hot)
print(classification_report(y_test,y_pred))

In [ ]:
clf.score(X_test_one_hot, y_test)

# MLP sin embeddings

In [ ]:
le = preprocessing.LabelEncoder()
le.fit(y_train.values)
Y_train_label = le.transform(y_train.values) 
y_train_one_hot = to_categorical(Y_train_label)
display(y_train_one_hot)

In [ ]:
y_test_label = le.transform(y_test.values) 
y_test_one_hot = to_categorical(y_test_label)
display(y_test_one_hot)

In [ ]:
model = Sequential() 
model.add(Dense(300, activation='relu', input_dim=351))
model.add(Dense(200, activation='relu'))
model.add(Dense(100, activation='relu'))
model.add(Dense(38)) 
optimizer = RMSprop(lr=0.001)
model.compile(optimizer=optimizer, loss='mean_squared_error', metrics= ['accuracy'])

In [ ]:
hist = model.fit(X_train_one_hot,y_train_one_hot, epochs=70,validation_data=(X_test_one_hot, y_test_one_hot), batch_size=120)

In [ ]:
plt.plot(hist.history['loss']) 
plt.plot(hist.history['val_loss']) 
plt.show()


plt.plot(hist.history['acc']) 
plt.plot(hist.history['val_acc']) 
plt.show()

## Pruebas grid search

In [ ]:
def create_model(neuronas1 = 25,neuronas2 =25,neuronas3 = 25):
           
    model = Sequential()
    activation='relu'
    init_mode='normal'
    dropout_rate=0.0
    weight_constraint=0
    model.add(Dense(neuronas1, activation=activation, input_dim=374,kernel_initializer=init_mode))
    model.add(Dense(neuronas2, activation=activation,kernel_initializer=init_mode))
    model.add(Dense(neuronas3, activation=activation,kernel_initializer=init_mode))
    model.add(Dropout(dropout_rate))
    model.add(Dense(38,kernel_initializer=init_mode))
    optimizer = RMSprop(lr=0.001)
    model.compile(loss='mean_squared_error', optimizer=optimizer, metrics=['accuracy'])
    return model

seed = 7
np.random.seed(seed)
model = KerasClassifier(build_fn=create_model, verbose=1,batch_size=120, epochs = 70)

In [ ]:
neuronas1 = [25,75,100,150,200,250,300,350,400]
neuronas2 = [25,75,100,150,200,250,300,350,400]
neuronas3 = [25,75,100,150,200,250,300,350,400]
param_grid = dict(neuronas1=neuronas1,neuronas2=neuronas2,neuronas3=neuronas3)
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=1, cv=2)
grid_result = grid.fit(X_train_one_hot, y_train_one_hot)

In [ ]:
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))